In [1]:
#Environment setup
import os
import sys
 
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/local/anaconda/bin/python" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/local/anaconda/bin/python"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.4-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

In [2]:
#SparkSession is the driver engine of the whole program
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
#read.csv will create a Spark DataFrame
items = spark.read.csv('data/items_Question6.csv',inferSchema=True,header=True)

In [5]:
items.show()

+-------+---------+---------------+
|item_id|item_name|item_unit_price|
+-------+---------+---------------+
|    101| Pendrive|          25.49|
|    102|   Laptop|         999.99|
|    103|Headphone|          99.99|
|    201|  Charger|          45.99|
|    202| Cable-1m|          12.99|
|    203|       OS|         299.99|
+-------+---------+---------------+



In [6]:
orders = spark.read.csv('data/orders_Question6.csv',inferSchema=True,header=True)

In [7]:
orders.show()

+--------+--------------------+-----------+------------+
|order_id|          order_date|customer_id|order_status|
+--------+--------------------+-----------+------------+
|   10001|2019-07-05 13:25:...|       2101|        Paid|
|   10002|2019-07-05 23:15:...|       2103|        Paid|
|   10003|2019-07-09 10:21:...|       2101|        Paid|
|   10004|2019-07-15 17:51:...|       2102|        Paid|
+--------+--------------------+-----------+------------+



In [8]:
order_items = spark.read.csv('data/order_items_Question6.csv',inferSchema=True,header=True)

In [9]:
order_items.show()

+--------------+--------+-------+--------+
|order_items_id|order_id|item_id|quantity|
+--------------+--------+-------+--------+
|        100001|   10001|    201|       3|
|        100002|   10001|    102|       1|
|        100003|   10002|    101|       4|
|        100004|   10002|    103|       2|
|        100005|   10003|    201|       3|
|        100006|   10004|    202|       5|
+--------------+--------+-------+--------+



## a.	Create the views orders_items, orders and items from the respective csv files. 

In [10]:
items.createOrReplaceTempView('itemsview')

In [11]:
orders.createOrReplaceTempView('ordersview')

In [12]:
order_items.createOrReplaceTempView('order_itemsview')

## b.	Count the number of orders per customer_id from the orders view

In [16]:
spark.sql('''
             select oi.customer_id,count(ot.order_id) as NumberOfOrders from order_itemsview  as ot
             join ordersview as oi on ot.order_id = oi.order_id
             group by oi.customer_id
             ''').show()

+-----------+--------------+
|customer_id|NumberOfOrders|
+-----------+--------------+
|       2102|             1|
|       2103|             2|
|       2101|             3|
+-----------+--------------+



In [22]:
spark.sql('''
             select it.item_name,count(ot.order_id) as NumberOfOrders from order_itemsview  as ot
             join itemsview as it on ot.item_id = it.item_id
             group by it.item_name
             ''').show()

+---------+--------------+
|item_name|NumberOfOrders|
+---------+--------------+
|Headphone|             1|
| Pendrive|             1|
|   Laptop|             1|
|  Charger|             2|
| Cable-1m|             1|
+---------+--------------+



## c.	Find the total amount of each order from the join of orders_items and items.

In [24]:
spark.sql('''
             select it.item_name,sum(it.item_unit_price) as TOTALAMOUNT,count(ot.order_id) as NumberOfOrders from order_itemsview  as ot
             join itemsview as it on ot.item_id = it.item_id
             group by it.item_name
             ''').show()

+---------+-----------+--------------+
|item_name|TOTALAMOUNT|NumberOfOrders|
+---------+-----------+--------------+
|Headphone|      99.99|             1|
| Pendrive|      25.49|             1|
|   Laptop|     999.99|             1|
|  Charger|      91.98|             2|
| Cable-1m|      12.99|             1|
+---------+-----------+--------------+

